# Improved Training

This week, we investigate some useful methods for improving the performance of neural nets by ensemble methods, where we train several nets on the same datapoints and allow them to "vote" on the correct classification or regression prediction. Another helpful approach is to monitor training carefully. Lastly, we will speeding up training with transfer learning, where we start with pre-trained models and try to specialize them for our datasets.

## Ensemble Methods

A common technique used with classification methods (both non-neural and neural) is to take an ensemble of models and combine them to make classification decisions. For example, we could run 5 neural nets, each with comparable accuracy overall, and classify each datapoint based on the majority vote of the 5 networks.

This almost always improves results compared to using just one net because different nets have unique talents and may make errors on different datapoints, but, assuming all the nets have good accuracy, they are typically correct, so these mistakes are frequently restricted to just a minority of models.

Let's bring in a few (mini) modern CNN achitectures we wrote in the past to use for some ensembling. Note that any architecture would work for the forthcoming experiments, but the following nets can run relatively quickly.

First, we import some things we need.

In [1]:
# basic packages
from imutils import paths
import glob
import h5py
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import progressbar
import random

# sklearn functions
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

# keras functions
from tensorflow.keras import backend
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import add
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical

### MiniVGGNet

In [2]:
# create a class for a mini version of VGGNet (Simonyan and Zisserman, 2015)
class MiniVGGNet:
    def build(height, width, depth, classes):
        # create the model and name it MiniVGGNet
        model = Sequential(name = 'MiniVGGNet')
                
        # convolutional layer with 32 3x3 feature maps
        model.add(Conv2D(32, (3, 3), padding = 'same', input_shape = (height, width, depth)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        # convolutional layer with 32 3x3 feature maps
        model.add(Conv2D(32, (3, 3), padding = 'same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        # 2x2 max pooling layer with stride 2x2
        model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))
        model.add(Dropout(0.25))
        
        # convolutional layer with 64 3x3 feature maps
        model.add(Conv2D(64, (3, 3), padding = 'same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        # convolutional layer with 64 3x3 feature maps
        model.add(Conv2D(64, (3, 3), padding = 'same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        # 2x2 max pooling layer with stride 2x2
        model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))
        model.add(Dropout(0.25))
        
        # flatten the activations from a square to a vector
        model.add(Flatten())
        
        # fully-connected layer
        model.add(Dense(512))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        
        # fully-connected layer with softmax classifier
        model.add(Dense(classes))
        model.add(Activation('softmax'))
        
        # return the model
        return model

### MiniGoogLeNet

In [3]:
class MiniGoogLeNet:
    def convolution_module(x, K, kX, kY, stride, channelsDim, padding="same"):
        # create a CONV -> BN -> RELU sequence
        x = Conv2D(K, (kX, kY), strides = stride, padding = padding)(x)
        x = BatchNormalization(axis = channelsDim)(x)
        x = Activation('relu')(x)
        
        # return the output
        return x
    
    def inception_module(x, numberOf1x1Kernels, numberOf3x3Kernels, channelsDim):
        # define two "parallel" convolutions of size 1x1 and 3x3 concatenated across the channels dimension
        convolution_1x1 = MiniGoogLeNet.convolution_module(x, numberOf1x1Kernels, 1, 1, (1, 1), channelsDim)
        convolution_3x3 = MiniGoogLeNet.convolution_module(x, numberOf3x3Kernels, 3, 3, (1, 1), channelsDim)
        x = concatenate([convolution_1x1, convolution_3x3], axis = channelsDim)
        
        return x
        
    def downsample_module(x, K, channelsDim):
        # define a CONV and POOL and then concatenate across the channels dimension
        convolution_3x3 = MiniGoogLeNet.convolution_module(x, K, 3, 3, (2, 2), channelsDim, padding = 'valid')
        pool = MaxPooling2D((3, 3), strides = (2, 2))(x)
        x = concatenate([convolution_3x3, pool], axis = channelsDim)
        
        return x
    
    def build(width, height, depth, classes):
        inputShape = (height, width, depth)
        channelsDim = -1
        
        if backend.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            channelsDim = 1
        
        # define the model input and first CONV module
        inputs = Input(shape = inputShape)
        x = MiniGoogLeNet.convolution_module(inputs, 96, 3, 3, (1, 1), channelsDim)
        
        # two inception modules followed by a downsample module
        x = MiniGoogLeNet.inception_module(x, 32, 32, channelsDim)
        x = MiniGoogLeNet.inception_module(x, 32, 48, channelsDim)
        x = MiniGoogLeNet.downsample_module(x, 80, channelsDim)
        
        # four inception modules followed by a downsample module
        x = MiniGoogLeNet.inception_module(x, 112, 48, channelsDim)
        x = MiniGoogLeNet.inception_module(x, 96, 64, channelsDim)
        x = MiniGoogLeNet.inception_module(x, 80, 80, channelsDim)
        x = MiniGoogLeNet.inception_module(x, 48, 96, channelsDim)
        x = MiniGoogLeNet.downsample_module(x, 96, channelsDim)
        
        # two inception modules followed by global POOL and dropout
        x = MiniGoogLeNet.inception_module(x, 176, 160, channelsDim)
        x = MiniGoogLeNet.inception_module(x, 176, 160, channelsDim)
        x = AveragePooling2D((7, 7))(x)
        x = Dropout(0.5)(x)
        
        # softmax classifier
        x = Flatten()(x)
        x = Dense(classes)(x)
        x = Activation('softmax')(x)
        
        # create a model
        model = Model(inputs, x, name='MiniGoogLeNet')
        
        # return the model
        return model

### ResNet

In [4]:
class ResNet:
    def residual_module(data, K, stride, channelsDim, reduce = False, reg = 0.0001, bnEpsilon = 0.00002, bnMomentum = 0.9):
        shortcut = data
        
        # 1x1 CONVs
        bn1 = BatchNormalization(axis = channelsDim, epsilon = bnEpsilon, momentum = bnMomentum)(data)
        act1 = Activation('relu')(bn1)
        conv1 = Conv2D(int(K * 0.25), (1, 1), use_bias = False, kernel_regularizer = l2(reg))(act1)
        
        # 3x3 CONVs
        bn2 = BatchNormalization(axis = channelsDim, epsilon = bnEpsilon, momentum = bnMomentum)(conv1)
        act2 = Activation('relu')(bn2)
        conv2 = Conv2D(int(K * 0.25), (3, 3), strides = stride, padding = 'same', use_bias = False, kernel_regularizer = l2(reg))(act2)
        
        # 1x1 CONVs
        bn3 = BatchNormalization(axis = channelsDim, epsilon = bnEpsilon, momentum = bnMomentum)(conv2)
        act3 = Activation('relu')(bn3)
        conv3 = Conv2D(K, (1, 1), use_bias = False, kernel_regularizer = l2(reg))(act3)
        
        # if we reduce the spatial size, apply a CONV layer to the shortcut
        if reduce:
            shortcut = Conv2D(K, (1, 1), strides = stride, use_bias = False, kernel_regularizer = l2(reg))(act1)
            
        # add the shortcut and the final CONV
        x = add([conv3, shortcut])
        
        return x
    
    def build(width, height, depth, classes, stages, filters, reg = 0.0001, bnEpsilon = 0.00002, bnMomentum = 0.9, dataset='cifar'):
        inputShape = (height, width, depth)
        channelsDim = -1
        
        if backend.image_data_format() == 'channels_first':
            inputShape = (depth, height, width)
            channelsDim = 1
            
        # set the input and apply BN
        inputs = Input(shape = inputShape)
        x = BatchNormalization(axis = channelsDim, epsilon = bnEpsilon, momentum = bnMomentum)(inputs)
        
        if dataset == 'cifar':
            # apply a single CONV layer
            x = Conv2D(filters[0], (3, 3), use_bias = False, padding = 'same',
                       kernel_regularizer = l2(reg))(x)
        
        # loop over the number of stages
        for counter in range(0, len(stages)):
            # initialize the stride
            if counter == 0:
                stride = (1, 1)
            else:
                stride = (2, 2)
                    
            # apply a residual module to reduce the spatial dimension of the image volume
            x = ResNet.residual_module(x, filters[counter + 1], stride, channelsDim, reduce = True, bnEpsilon = bnEpsilon, bnMomentum = bnMomentum)
            
            # loop over the number of layers in the current stage
            for j in range(0, stages[counter] - 1):
                # apply a residual module
                x = ResNet.residual_module(x, filters[counter + 1], (1, 1), channelsDim, bnEpsilon = bnEpsilon, bnMomentum = bnMomentum)
                    
        # apply BN -> ACT -> POOL
        x = BatchNormalization(axis = channelsDim, epsilon = bnEpsilon, momentum = bnMomentum)(x)
        x = Activation('relu')(x)
        x = AveragePooling2D((8, 8))(x)
        
        # softmax classifier
        x = Flatten()(x)
        x = Dense(classes, kernel_regularizer = l2(reg))(x)
        x = Activation('softmax')(x)
        
        # create the model
        model = Model(inputs, x, name = 'ResNet')
        
        # return the model
        return model

### Training Ensembles

Now, let's look at some ensembling methods. In the simplest case, we train several nets and average the classifications at the end. If the nets have similar performance, but make mistakes on *different* examples, this approach improves performance in most cases.

**Quick GPU Check**: Before we start training models, let's check our GPU resources. If you have a GPU set up to work with TensorFlow, its name will be output and it will be used in training. If not, none will be output and your training will use your CPU.

In [2]:
import tensorflow as tf
from tensorflow.python.client import device_lib

numGPUs = len(tf.config.experimental.list_physical_devices('GPU'))

print("Num GPUs Available: ", numGPUs)

if numGPUs > 0:
    print(tf.test.gpu_device_name())
    print(device_lib.list_local_devices()[1].physical_device_desc)

Num GPUs Available:  1
/device:GPU:0
device: 0, name: GeForce RTX 2070, pci bus id: 0000:04:00.0, compute capability: 7.5


### Training with the Same Hyperparameters Multiple Times

Next, let's train several MiniVGGNets on CIFAR-10.

In [6]:
tf.keras.backend.clear_session()

# load cifar10 data
((trainX, trainY), (testX, testY)) = cifar10.load_data()
labelNames = ["airplane", "automobile", "bird", "cat", "deer", 'dog', "frog", "horse", "ship", "truck"]

# preprocess data
trainX = trainX.astype('float')/255.0
testX = testX.astype('float')/255.0
trainY = to_categorical(trainY, 10)
testY = to_categorical(testY, 10)

# create an image generator for data augmentation with random shifting, rotation, and horizontal flips
aug = ImageDataGenerator(rotation_range = 10, width_shift_range = 0.1, height_shift_range = 0.1, horizontal_flip = True, fill_mode = 'nearest')

In [7]:
tf.keras.backend.clear_session()

numberOfModels = 5
epochs = 10

for i in range(numberOfModels):
    print('Net', i, 'is being trained...')
    
    # choose the optimizer
    #opt = SGD(lr = 0.01, decay = 0.1 / epochs, momentum = 0.9, nesterov = True)
    opt = Adam()
    
    # compile the model
    model = MiniVGGNet.build(32, 32, 3, 10)
    model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])
    
    # train the model
    H = model.fit(aug.flow(trainX, trainY, batch_size = 64), validation_data = (testX, testY), epochs = epochs, steps_per_epoch = len(trainX) // 64, verbose = 1)
    
    # save the model
    p = ['models1', 'model_{}.model'.format(i)]
    model.save(os.path.sep.join(p))
    
    # evaluate the network
    predictions = model.predict(testX, batch_size=64)
    report = classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=labelNames)
    
    # save the classification report to file
    p = ['output1', 'model_{}.txt'.format(i)]
    f = open(os.path.sep.join(p), "w")
    f.write(report)
    f.close()
    
    # plot the training loss and accuracy
    p = ['output1', 'model_{}.png'.format(i)]
    plt.style.use('ggplot')
    plt.figure()
    plt.plot(np.arange(0, epochs), H.history['loss'], label = 'train_loss')
    plt.plot(np.arange(0, epochs), H.history['val_loss'], label = 'val_loss')
    plt.plot(np.arange(0, epochs), H.history['accuracy'], label = 'train_acc')
    plt.plot(np.arange(0, epochs), H.history['val_accuracy'], label = 'val_acc')
    
    # add labels and legend
    plt.title('Training Loss and Accuracy for model {}'.format(i))
    plt.xlabel('Epoch #')
    plt.ylabel('Loss/Accuracy')
    plt.legend()
    
    # save graphs
    plt.savefig(os.path.sep.join(p))
    plt.close()

Net 0 is being trained...
  ...
    to  
  ['...']
Train for 781 steps, validate on 10000 samples
Epoch 1/10
781/781 [==============================] - 84s 107ms/step - loss: 1.6116 - accuracy: 0.4564 - val_loss: 1.2383 - val_accuracy: 0.5816
Epoch 2/10
781/781 [==============================] - 22s 28ms/step - loss: 1.1182 - accuracy: 0.6013 - val_loss: 1.0832 - val_accuracy: 0.6394
Epoch 3/10
781/781 [==============================] - 22s 29ms/step - loss: 0.9926 - accuracy: 0.6504 - val_loss: 1.1733 - val_accuracy: 0.6197
Epoch 4/10
781/781 [==============================] - 22s 28ms/step - loss: 0.9169 - accuracy: 0.6774 - val_loss: 0.9389 - val_accuracy: 0.6803
Epoch 5/10
781/781 [==============================] - 22s 29ms/step - loss: 0.8591 - accuracy: 0.7008 - val_loss: 0.9506 - val_accuracy: 0.6894
Epoch 6/10
781/781 [==============================] - 23s 29ms/step - loss: 0.8172 - accuracy: 0.7133 - val_loss: 0.9262 - val_accuracy: 0.7068
Epoch 7/10
781/781 [=================

Next, we test the ensemble.

In [8]:
# construct the path used to collect the models then initialize the
# models list
modelPaths = os.path.sep.join(['models1', '*.model'])
modelPaths = list(glob.glob(modelPaths))
models = []

# loop over the model paths, loading the model, and adding it to
# the list of models
for (i, modelPath) in enumerate(modelPaths):
	print('Loading model {}/{}'.format(i + 1, len(modelPaths)))
	models.append(load_model(modelPath))

# initialize the list of predictions
print('Evaluating ensemble...')
predictions = []

# loop over the models
for model in models:
	# use the current model to make predictions on the testing data,
	# then store these predictions in the aggregate predictions list
	predictions.append(model.predict(testX, batch_size=64))

# average the probabilities across all model predictions, then show
# a classification report
predictions = np.average(predictions, axis=0)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=labelNames))

Loading model 1/5
Loading model 2/5
Loading model 3/5
Loading model 4/5
Loading model 5/5
Evaluating ensemble...
              precision    recall  f1-score   support

    airplane       0.81      0.86      0.83      1000
  automobile       0.90      0.93      0.91      1000
        bird       0.79      0.68      0.73      1000
         cat       0.74      0.51      0.60      1000
        deer       0.74      0.81      0.77      1000
         dog       0.86      0.57      0.69      1000
        frog       0.64      0.96      0.77      1000
       horse       0.85      0.85      0.85      1000
        ship       0.92      0.89      0.90      1000
       truck       0.83      0.93      0.87      1000

    accuracy                           0.80     10000
   macro avg       0.81      0.80      0.79     10000
weighted avg       0.81      0.80      0.79     10000



Here, we actually ran the network with the very same hyperparameters 5 times and found that the best ones performed at around **77%**, but the ensemble of all 5 averaged together reseults in a higher accuracy rate of **80%**!

Mathematically, what has happened is likely that the different nets converged to *different* local minima so that, while their individual accuracy rates were all similar, the different nets were, apparently misclassifying *different* test examples, resulting in the average classification of all the models being correct more frequently than any individual net.

### Ensemble Training with Different Nets

It stands to reason that nets that are more significantly different are more likely to make significantly different classification mistakes since they may work in very different ways. For example, our past GoogLeNet and ResNet experiments had about 90% success on CIFAR-10, but they are very different architectures. Let's try to create an ensemble of those!

In [6]:
tf.keras.backend.clear_session()
    
# load cifar10 data
((trainX, trainY), (testX, testY)) = cifar10.load_data()
trainX = trainX.astype('float')
testX = testX.astype('float')
labelNames = ["airplane", "automobile", "bird", "cat", "deer", 'dog', "frog", "horse", "ship", "truck"]

# preprocess data
mean = np.mean(trainX, axis = 0)
trainX -= mean
testX -= mean
trainY = to_categorical(trainY, 10)
testY = to_categorical(testY, 10)

# create an image generator for data augmentation with random shifting, rotation, and horizontal flips
aug = ImageDataGenerator(rotation_range = 10, width_shift_range = 0.1, height_shift_range = 0.1, horizontal_flip = True, fill_mode = 'nearest')

In [9]:
tf.keras.backend.clear_session()

numberOfModels = 2
epochs = 100

opt = Adam()

for i in range(numberOfModels):
    print('GoogLeNet', i, 'is being trained...')
    
    # compile the model
    model = MiniGoogLeNet.build(width = 32, height = 32, depth = 3, classes = 10)
    model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics=['accuracy'])
    
    # train the model
    H = model.fit(aug.flow(trainX, trainY, batch_size = 64), validation_data = (testX, testY), epochs = epochs,
                  steps_per_epoch = len(trainX) // 64, verbose = 1)
    
    # save the model
    p = ['models2', 'model_{}.model'.format(i)]
    model.save(os.path.sep.join(p))
    
    # evaluate the network
    predictions = model.predict(testX, batch_size=64)
    report = classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=labelNames)
    
    # save the classification report to file
    p = ['output2', 'model_{}.txt'.format(i)]
    f = open(os.path.sep.join(p), 'w')
    f.write(report)
    f.close()
    
    # plot the training loss and accuracy
    p = ['output2', 'model_{}.png'.format(i)]
    plt.style.use('ggplot')
    plt.figure()
    plt.plot(np.arange(0, epochs), H.history['loss'], label = 'train_loss')
    plt.plot(np.arange(0, epochs), H.history['val_loss'], label = 'val_loss')
    plt.plot(np.arange(0, epochs), H.history['accuracy'], label = 'train_acc')
    plt.plot(np.arange(0, epochs), H.history['val_accuracy'], label = 'val_acc')
    
    # add labels and legend
    plt.title('Training Loss and Accuracy for model {}'.format(i))
    plt.xlabel('Epoch #')
    plt.ylabel('Loss/Accuracy')
    plt.legend()
    
    # save graphs
    plt.savefig(os.path.sep.join(p))
    plt.close()
    

for i in range(numberOfModels, 2*numberOfModels+1):
    print('ResNet', i, 'is being trained...')
    
    # compile the model
    model = ResNet.build(32, 32, 3, 10, (9, 9, 9), (64, 64, 128, 256), reg=0.0005)
    model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics=['accuracy'])
    
    # train the model
    H = model.fit(aug.flow(trainX, trainY, batch_size = 64), validation_data = (testX, testY), epochs = epochs,
                  steps_per_epoch = len(trainX) // 64, verbose = 1)
    
    # save the model
    p = ['models2', 'model_{}.model'.format(i)]
    model.save(os.path.sep.join(p))
    
    # evaluate the network
    predictions = model.predict(testX, batch_size=64)
    report = classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=labelNames)
    
    # save the classification report to file
    p = ['output2', 'model_{}.txt'.format(i)]
    f = open(os.path.sep.join(p), 'w')
    f.write(report)
    f.close()
    
    # plot the training loss and accuracy
    p = ['output2', 'model_{}.png'.format(i)]
    plt.style.use('ggplot')
    plt.figure()
    plt.plot(np.arange(0, epochs), H.history['loss'], label = 'train_loss')
    plt.plot(np.arange(0, epochs), H.history['val_loss'], label = 'val_loss')
    plt.plot(np.arange(0, epochs), H.history['accuracy'], label = 'train_acc')
    plt.plot(np.arange(0, epochs), H.history['val_accuracy'], label = 'val_acc')
    
    # add labels and legend
    plt.title('Training Loss and Accuracy for model {}'.format(i))
    plt.xlabel('Epoch #')
    plt.ylabel('Loss/Accuracy')
    plt.legend()
    
    # save graphs
    plt.savefig(os.path.sep.join(p))
    plt.close()

GoogLeNet 0 is being trained...
  ...
    to  
  ['...']
Train for 781 steps, validate on 10000 samples
Epoch 1/100
781/781 [==============================] - 31s 40ms/step - loss: 1.3488 - accuracy: 0.5128 - val_loss: 2.0277 - val_accuracy: 0.4694
Epoch 2/100
781/781 [==============================] - 29s 37ms/step - loss: 0.8970 - accuracy: 0.6853 - val_loss: 0.8755 - val_accuracy: 0.6964
Epoch 3/100
781/781 [==============================] - 28s 36ms/step - loss: 0.7277 - accuracy: 0.7491 - val_loss: 0.7028 - val_accuracy: 0.7612
Epoch 4/100
781/781 [==============================] - 29s 37ms/step - loss: 0.6233 - accuracy: 0.7872 - val_loss: 0.8016 - val_accuracy: 0.7301
Epoch 5/100
781/781 [==============================] - 29s 37ms/step - loss: 0.5506 - accuracy: 0.8116 - val_loss: 0.7213 - val_accuracy: 0.7692
Epoch 6/100
781/781 [==============================] - 29s 37ms/step - loss: 0.5000 - accuracy: 0.8286 - val_loss: 0.8530 - val_accuracy: 0.7382
Epoch 7/100
781/781 [=====

In [10]:
# construct the path used to collect the models then initialize the
# models list
modelPaths = os.path.sep.join(['models2', '*.model'])
modelPaths = list(glob.glob(modelPaths))
models = []

# loop over the model paths, loading the model, and adding it to
# the list of models
for (i, modelPath) in enumerate(modelPaths):
	print('Loading model {}/{}'.format(i + 1, len(modelPaths)))
	models.append(load_model(modelPath))

# initialize the list of predictions
print('Evaluating ensemble...')
predictions = []

# loop over the models
for model in models:
	# use the current model to make predictions on the testing data,
	# then store these predictions in the aggregate predictions list
	predictions.append(model.predict(testX, batch_size = 64))

# average the probabilities across all model predictions, then show
# a classification report
predictions = np.average(predictions, axis = 0)
print(classification_report(testY.argmax(axis = 1), predictions.argmax(axis = 1), target_names = labelNames))

Loading model 1/5
Loading model 2/5
Loading model 3/5
Loading model 4/5
Loading model 5/5
Evaluating ensemble...
              precision    recall  f1-score   support

    airplane       0.94      0.95      0.95      1000
  automobile       0.95      0.98      0.97      1000
        bird       0.91      0.90      0.91      1000
         cat       0.85      0.86      0.86      1000
        deer       0.93      0.93      0.93      1000
         dog       0.92      0.85      0.88      1000
        frog       0.90      0.98      0.94      1000
       horse       0.97      0.95      0.96      1000
        ship       0.97      0.96      0.96      1000
       truck       0.97      0.95      0.96      1000

    accuracy                           0.93     10000
   macro avg       0.93      0.93      0.93     10000
weighted avg       0.93      0.93      0.93     10000



### Snapshot Ensembles

Snapshot ensembles (Huang, et. al., 2017) train a single net in such a way that it repeatedly converges to different local minima, save each state of the model, and use (a subset of) those saved states to build an ensemble. While there may be more risk of this one net landing in similar local minima when training one net compared to training several unrelated nets, it can be dramatically cheaper computationally to ensemble in this way.

One way of accomplishing this is to use a cyclic learning rate that starts high and anneals to smaller values until the net settles, saves the state of the model, and then returns to a high learning rate to repeat the cycle. This way, it settles down to a state we will use in the ensemble, but then, a high learning rate will let it jump out of the local minimum it has hopefully reached and continue to a new one. This way, our learning algorithms explore more of the parameter space and capture multiple local minima.

The immediate question may be, "Why not just train a new model from a random initialization each time?" But, what tends to happen is that most training time occurs when a net is trying to reach its first local minimum. Routing from there to a new one has been shown to be much cheaper under the right circumstances.

To accomplish this, we simply need to write a learning rate scheduler that applies a cyclical learning rate of the form (Loschilov and Hutter, 2016):

$$\alpha(t) = f\left(\text{mod}\left(t - 1, \left\lceil \frac{T}{M}\right\rceil\right)\right),$$

where $t$ is the iteration number, $T$ is the total number of training epochs, $M$ is the number of snapshots we will capture, and $f$ is a monotonically decreasing function. So here, we specify how many epochs to train and the number of snapshots we want to take, and then the learning rate cycles every $\frac{\text{number of training epochs}}{\text{number of snapshots}}$ number of epochs.

Loschilov and Hutter proposed a shifted cosine function of the form

$$\alpha(t)=\frac{\alpha_0}{2}\left(\cos\left(\frac{\pi\text{ mod}(t-1,\lceil T/M\rceil)}{\lceil T/M \rceil}\right)+1\right),$$

where $\alpha_0$ is the initial learning rate.

In [8]:
tf.keras.backend.clear_session()

# load cifar10 data
((trainX, trainY), (testX, testY)) = cifar10.load_data()
labelNames = ["airplane", "automobile", "bird", "cat", "deer", 'dog', "frog", "horse", "ship", "truck"]

# preprocess data
trainX = trainX.astype('float')/255.0
testX = testX.astype('float')/255.0
trainY = to_categorical(trainY, 10)
testY = to_categorical(testY, 10)

# create an image generator for data augmentation with random shifting, rotation, and horizontal flips
aug = ImageDataGenerator(rotation_range = 10, width_shift_range = 0.1, height_shift_range = 0.1, horizontal_flip = True, fill_mode = 'nearest')

In [9]:
tf.keras.backend.clear_session()

models = 3
initialLearningRate = 0.2
epochs = 60

# code for a learning rate scheduler
def shiftedCosineLearning(epoch):
    maxEpochs = epochs
    baseLearningRate = initialLearningRate

    alpha = (initialLearningRate/2)*(np.cos(np.pi*np.mod(epoch - 1, np.ceil(epochs/models))/np.ceil(epochs/models)) + 1)
    
    # return the learning rate
    return alpha

callbacks = [LearningRateScheduler(shiftedCosineLearning)]
    
# choose the optimizer
opt = SGD(lr = initialLearningRate)
#pt = Adam()
    
# compile the model
model = MiniVGGNet.build(32, 32, 3, 10)
model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])
    
# train the model
checkpoint = ModelCheckpoint('model{epoch:08d}.model', period=np.ceil(epochs/models)) 
H = model.fit(aug.flow(trainX, trainY, batch_size = 64), validation_data = (testX, testY), epochs = epochs,
              callbacks = callbacks, steps_per_epoch = len(trainX) // 64, verbose = 1)
    
# evaluate the network
predictions = model.predict(testX, batch_size=64)
report = classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=labelNames)

Train for 781 steps, validate on 10000 samples
Epoch 1/60
 83/781 [==>...........................] - ETA: 6:16 - loss: 3.3040 - accuracy: 0.1431

KeyboardInterrupt: 

Next, we test the ensemble.

In [7]:
# THERE IS SOME PREPROCESSING ERROR HERE IN THE LAST STEP...

# construct the path used to collect the models then initialize the
# models list
modelPaths = os.path.sep.join(['*.model'])
modelPaths = list(glob.glob(modelPaths))
models = []

# loop over the model paths, loading the model, and adding it to
# the list of models
for (i, modelPath) in enumerate(modelPaths):
	print('Loading model {}/{}'.format(i + 1, len(modelPaths)))
	models.append(load_model(modelPath))

# initialize the list of predictions
print('Evaluating ensemble...')
predictions = []

# loop over the models
for model in models:
	# use the current model to make predictions on the testing data,
	# then store these predictions in the aggregate predictions list
	predictions.append(model.predict(testX, batch_size=64))

# average the probabilities across all model predictions, then show
# a classification report
predictions = np.average(predictions, axis=0)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=labelNames))

Evaluating ensemble...


C:\Users\Ryan\anaconda3\envs\DL\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\Users\Ryan\anaconda3\envs\DL\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


NameError: name 'testY' is not defined

## Transfer Learning

Transfer learning is where we take nets pre-trained on huge datasets like ImageNet, load the weights, and use those as starting points for training on a new dataset. The idea is that knowledge learning about one dataset, if it is somewhat related to your dataset, can *transfer* to knowledge of your dataset with only partial training.

For example, a neural net that is effective at classifying words spoken in English might be effective for learning words spoken in Spanish without totally starting from scratch with training and hyperparameter tuning. After all, both languages have roughly the same alphabet and many similarities in pronunciation of letters.

One approach takes a pre-trained convolutional neural net containing all the parameters successful at classifying the intended dataset, remove the fully-connected layers at the end, and feed your *different* dataset through the net to the end of the last pooling layer. Then, treat the outputs from each input as a new dataset that has been preprocessed by ths pre-trained CNN. Lastly, apply another classifier to this dataset.

Another common approach is to take the same kind of pre-trained CNN and re-initialize the weights of the fully-connected layers at the end. Then, "freeze" all the parameters before the fully-connected layers. Last, train the parameters of the last few layers with a small learning rate on the new dataset, but with one caveat: as backpropagation moves backward through the network determining weight updates, it stops when it reaches the frozen layers and does not adjust those weights at all.

Let's learn how to implement both approaches!

## HDF5

If we want to work with huge pre-trained neural nets like VGG19 or other deep CNNs, storing them takes far more space than our RAM is likely to support, so we need to store them on HDD/SDDs in an efficient way. Keras's model format is pretty large, but HDF5 is a good data format for this, but we need some code to be able to interface with this format, which we write below.

In [2]:
class HDF5DatasetWriter:
    def __init__(self, dims, outputPath, dataKey = 'images', bufferSize = 1000):
        # check if outputpath exists
        if os.path.exists(outputPath):
            raise ValueError('The supplied `outputPath` already exists and cannot be overwritten. Delete '
                            ' the file manually before continuing.', outputPath)
            
        # open the HDF5 database for writing and create two datasets: one to store the images/features and
        # one to store the labels
        self.db = h5py.File(outputPath, 'w')
        self.data = self.db.create_dataset(dataKey, dims, dtype = 'float')
        self.labels = self.db.create_dataset('labels', (dims[0],), dtype = 'float')
        
        # store the buffer size and initialize the buffer and index
        self.bufferSize = bufferSize
        self.buffer = {'data': [], 'labels': []}
        self.index = 0
        
    def add(self, rows, labels):
        # add the rows and labels to the buffer
        self.buffer['data'].extend(rows)
        self.buffer['labels'].extend(labels)
        
        # check if the buffer needs to be flushed to disk
        if len(self.buffer['data']) >= self.bufferSize:
            self.flush()
            
    def flush(self):
        # write the buffer to disk and reset buffer
        i = self.index + len(self.buffer['data'])
        self.data[self.index:i] = self.buffer['data']
        self.labels[self.index:i] = self.buffer['labels']
        
        self.index = i
        self.buffer = {'data': [], 'labels': []}
        
    def storeClassLabels(self, classLabels):
        # create a dataset to store class label names, then store them
        dt = h5py.special_dtype(vlen = str)
        labelSet = self.db.create_dataset('label_names', (len(classLabels),), dtype = dt)
        labelSet[:] = classLabels
        
    def close(self):
        # flush entries to disk if needed
        if len(self.buffer['data']) > 0:
            self.flush()
            
        # close the dataset
        self.db.close()

## Feature extraction

Let's write some code to extract features from an arbitrary image dataset.

In [3]:
def extractFeatures(batch_size, dataset, output, buffer_size = 1000):

    bs = batch_size
    
    # grab the list of images that we'll be describing then randomly
    # shuffle them to allow for easy training and testing splits via
    # array slicing during training time
    print("[INFO] loading images...")
    imagePaths = list(paths.list_images(dataset))
    random.shuffle(imagePaths)

    # extract the class labels from the image paths then encode the
    # labels
    labels = [p.split(os.path.sep)[-2] for p in imagePaths]
    le = LabelEncoder()
    labels = le.fit_transform(labels)

    # load the VGG16 network
    print("[INFO] loading network...")
    model = VGG16(weights="imagenet", include_top=False)

    # initialize the HDF5 dataset writer, then store the class label
    # names in the dataset
    dataset = HDF5DatasetWriter((len(imagePaths), 512 * 7 * 7),
        output, dataKey="features", bufferSize=buffer_size)
    dataset.storeClassLabels(le.classes_)

    # initialize the progress bar
    widgets = ["Extracting Features: ", progressbar.Percentage(), " ",
        progressbar.Bar(), " ", progressbar.ETA()]
    pbar = progressbar.ProgressBar(maxval=len(imagePaths),
        widgets=widgets).start()

    # loop over the images in batches
    for i in np.arange(0, len(imagePaths), bs):
        # extract the batch of images and labels, then initialize the
        # list of actual images that will be passed through the network
        # for feature extraction
        batchPaths = imagePaths[i:i + bs]
        batchLabels = labels[i:i + bs]
        batchImages = []

        # loop over the images and labels in the current batch
        for (j, imagePath) in enumerate(batchPaths):
            # load the input image using the Keras helper utility
            # while ensuring the image is resized to 224x224 pixels
            image = load_img(imagePath, target_size=(224, 224))
            image = img_to_array(image)

            # preprocess the image by (1) expanding the dimensions and
            # (2) subtracting the mean RGB pixel intensity from the
            # ImageNet dataset
            image = np.expand_dims(image, axis=0)
            image = imagenet_utils.preprocess_input(image)

            # add the image to the batch
            batchImages.append(image)

        # pass the images through the network and use the outputs as
        # our actual features
        batchImages = np.vstack(batchImages)
        features = model.predict(batchImages, batch_size=bs)

        # reshape the features so that each image is represented by
        # a flattened feature vector of the `MaxPooling2D` outputs
        features = features.reshape((features.shape[0], 512 * 7 * 7))

        # add the features and labels to our HDF5 dataset
        dataset.add(features, batchLabels)
        pbar.update(i)

    # close the dataset
    dataset.close()
    pbar.finish()

### Extracting Features from Datasets

In [4]:
extractFeatures(32, '../datasets/animals/images', '../datasets/animals/hdf5/features.hdf5', 1000)

[INFO] loading images...
[INFO] loading network...


Extracting Features: 100% |#####################################| Time: 0:00:32


In [5]:
extractFeatures(32, '../datasets/caltech-101/images', '../datasets/caltech-101/hdf5/features.hdf5', 1000)

[INFO] loading images...
[INFO] loading network...


Extracting Features: 100% |#####################################| Time: 0:01:37


In [6]:
extractFeatures(32, '../datasets/flowers17/images', '../datasets/flowers17/hdf5/features.hdf5', 1000)

[INFO] loading images...
[INFO] loading network...


Extracting Features: 100% |#####################################| Time: 0:00:16


### Training a Classifier on Extracted Features

We have used a VGG16 net pre-trained on ImageNet and used it to extract features from three *different* datasets. We will now train a simple classifier on this new dataset of features extracted by the VGG16 net and see if the learning can actually be transferred.

In [ ]:
db = h5py